In [1]:
import tensorflow as tf
import tensorflow_hub as hub

/content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
[main c625735] Still testing google colab
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 398 bytes | 398.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/Rohith04MVK/Object-Detection-Website.git
   c25714a..c625735  main -> main
